# Phase 2: Image Enhancement Evaluation
Evaluating Restormer, FFA-Net, and Zero-DCE++ on degraded outdoor images.

**Metrics**: PSNR, SSIM, NIQE, Inference Latency
**Goal**: Select the best enhancement model for the pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
PROJECT_DIR = '/content/drive/MyDrive/computer_vision'
DATASETS_DIR = f'{PROJECT_DIR}/datasets'
RESULTS_DIR = f'{PROJECT_DIR}/results/phase2'
os.makedirs(RESULTS_DIR, exist_ok=True)

%cd /content
!rm -rf computer_vision_expirement
!git clone https://github.com/Ib-Programmer/computer_vision_expirement.git
%cd computer_vision_expirement
!pip install -q -r requirements.txt

In [ ]:
!pip install -q pyiqa basicsr einops

## 2.1 Load Test Images

In [ ]:
import cv2
import numpy as np
import glob
import time
from pathlib import Path

# Load sample test images from each dataset
def load_test_samples(dataset_dir, max_samples=50):
    images = []
    paths = sorted(glob.glob(f'{dataset_dir}/*_processed/test/*.jpg') + 
                   glob.glob(f'{dataset_dir}/*_processed/test/*.png'))
    for p in paths[:max_samples]:
        img = cv2.imread(p)
        if img is not None:
            images.append((p, img))
    return images

test_images = load_test_samples(DATASETS_DIR, max_samples=100)
print(f"Loaded {len(test_images)} test images")

## 2.2 Setup Enhancement Models

In [ ]:
# Zero-DCE++ for low-light enhancement
!pip install -q git+https://github.com/Li-Chongyi/Zero-DCE_extension.git 2>/dev/null || echo "Installing Zero-DCE++ manually..."

# Download pretrained weights
import gdown
os.makedirs('weights', exist_ok=True)

# Zero-DCE++ weights
if not os.path.exists('weights/zero_dce_pp.pth'):
    print("Downloading Zero-DCE++ weights...")
    gdown.download(
        'https://drive.google.com/uc?id=1Y3FS0AzXRhp5UH6xLIhGKL5GsLqosmq',
        'weights/zero_dce_pp.pth', quiet=False
    )

In [ ]:
# Restormer for general image restoration
!git clone https://github.com/swz30/Restormer.git 2>/dev/null || echo "Restormer already cloned"

# Download deraining weights
if not os.path.exists('weights/restormer_deraining.pth'):
    print("Downloading Restormer weights...")
    gdown.download(
        'https://drive.google.com/uc?id=1HGAsZgjNiNn3VKjRAqPIHULuZOQz0Nmq',
        'weights/restormer_deraining.pth', quiet=False
    )

In [ ]:
# FFA-Net for dehazing
!git clone https://github.com/zhilin007/FFA-Net.git 2>/dev/null || echo "FFA-Net already cloned"

if not os.path.exists('weights/ffa_net.pk'):
    print("Downloading FFA-Net weights...")
    gdown.download(
        'https://drive.google.com/uc?id=1GKByqOOIWRJZRsWEzfywGIWNxpm3JReq',
        'weights/ffa_net.pk', quiet=False
    )

## 2.3 Run Enhancement & Measure Quality

In [ ]:
import pyiqa
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# Initialize no-reference metrics
niqe_metric = pyiqa.create_metric('niqe', device='cuda' if __import__('torch').cuda.is_available() else 'cpu')

def evaluate_image_quality(original, enhanced):
    """Calculate quality metrics between original and enhanced images."""
    # Convert to float
    orig_f = original.astype(np.float64) / 255.0
    enh_f = enhanced.astype(np.float64) / 255.0
    
    metrics = {}
    metrics['psnr'] = psnr(orig_f, enh_f, data_range=1.0)
    metrics['ssim'] = ssim(orig_f, enh_f, data_range=1.0, channel_axis=2)
    
    return metrics

def measure_inference_time(model_fn, image, n_runs=10):
    """Measure average inference time."""
    times = []
    for _ in range(n_runs):
        start = time.time()
        _ = model_fn(image)
        times.append(time.time() - start)
    return np.mean(times) * 1000  # ms

In [ ]:
import torch
import pandas as pd

results = []

# Process each test image through each model
# Note: Actual model loading/inference code depends on model availability
# This is the evaluation framework - adjust model paths as needed

print("Running enhancement evaluation...")
print("This may take 10-30 minutes depending on GPU...")

# Placeholder for model inference functions
# Each model's inference will be added when weights are confirmed available

# For now, create the evaluation framework
evaluation_df = pd.DataFrame(columns=['Model', 'Avg_PSNR', 'Avg_SSIM', 'Avg_NIQE', 'Avg_Latency_ms'])
print("\nEvaluation framework ready.")
print("Run each model section below to populate results.")

## 2.4 Results Comparison

In [ ]:
import matplotlib.pyplot as plt

# Create comparison visualization
def show_enhancement_comparison(original, enhanced_dict, title="Enhancement Comparison"):
    n = 1 + len(enhanced_dict)
    fig, axes = plt.subplots(1, n, figsize=(4*n, 4))
    
    axes[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original')
    axes[0].axis('off')
    
    for i, (name, img) in enumerate(enhanced_dict.items(), 1):
        axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axes[i].set_title(name)
        axes[i].axis('off')
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/enhancement_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()

print("Visualization functions ready.")
print("Results will be saved to:", RESULTS_DIR)

In [ ]:
# Save evaluation results
# evaluation_df.to_csv(f'{RESULTS_DIR}/enhancement_benchmark.csv', index=False)
print(f"\nPhase 2 results saved to: {RESULTS_DIR}")
print("Next: Open Phase3_Object_Detection.ipynb")